<a href="https://colab.research.google.com/github/oceam/agri-image/blob/main/codes/VegAnn_tuturial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a tuturial code for the VggAnn (https://github.com/simonMadec/VegAnn) deep learning networks

> **VegAnn**   
> Vegetation Annotation of a large multi-crop RGB Dataset acquired under diverse conditions for image semantic segmentation